<a href="https://colab.research.google.com/github/Dias-lezdo/llama-model/blob/main/llama3_1_rag_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.7 MB/s

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from google.colab import userdata
GROQ_API_KEY = "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"

# GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [4]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["/content/Sample-filled-in-MR.pdf"])
documents = reader.load_data()

In [ ]:
len(documents)

2

In [5]:
documents[0].metadata

{'page_label': '1',
 'file_name': 'Sample-filled-in-MR.pdf',
 'file_path': '/content/Sample-filled-in-MR.pdf',
 'file_type': 'application/pdf',
 'file_size': 309139,
 'creation_date': '2024-10-29',
 'last_modified_date': '2024-10-29'}

In [6]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
len(nodes)

2

In [7]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'Sample-filled-in-MR.pdf',
 'file_path': '/content/Sample-filled-in-MR.pdf',
 'file_type': 'application/pdf',
 'file_size': 309139,
 'creation_date': '2024-10-29',
 'last_modified_date': '2024-10-29'}

In [8]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
llm = Groq(model="llama-3.1-8b-instant", api_key=GROQ_API_KEY)

In [12]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [13]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [15]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

In [16]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [17]:
query_engine = index.as_query_engine(service_context=service_context)

In [23]:
query = "Does Mr. Tan have the mental capacity for personal welfare and property decisions?"
resp = query_engine.query(query)
print(resp.response)

Based on the information provided, it appears that Mr. Tan's mental capacity is significantly impaired. He was unable to accurately identify the date, day, and time, and was also unable to answer basic questions about his personal information, such as his age and address. Additionally, he was unable to tell the time from looking at a watch, which suggests a significant cognitive impairment.

Furthermore, the clinical impression was that Mr. Tan was manifesting behavioural and psychological symptoms secondary to Dementia, which is a condition that affects a person's mental capacity. The fact that he was unable to bathe or use the toilet on his own, and required assistance with daily activities, also suggests a significant impairment in his mental capacity.

Given these findings, it is likely that Mr. Tan does not have the mental capacity for personal welfare and property decisions.
